In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No Api Key was found')
elif not api_key.startswith('sk-proj-'):
    print("An api key was found, but it doesnt start with sk-proj")
elif api_key.strip() != api_key:
    print("An api key was found, but it might have space in the first or end")
else:
    print("Api key found and looks good so far!")

Api key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
message = "Hello GPT, this is my first message"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! How can I assist you today?


In [5]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
github = Website("https://github.com/Fikriraihan")
print(github.title)
print(github.text)

Fikriraihan · GitHub
Skip to content
Navigation Menu
Toggle navigation
Sign in
Appearance settings
Product
GitHub Copilot
Write better code with AI
GitHub Models
New
Manage and compare prompts
GitHub Advanced Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
Why GitHub
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries
View all solutions
Resources
Topics
AI
DevOps
Security
Software Development
View all
Explore
Learning Pathways
Events & Webinars
Ebooks & Whitepapers
Customer Stories
Partners
Executive Insights
Open Source
GitHub Sponsors
Fund open source developers
The ReadME Proj

In [12]:
system_prompt = "You are a skilled GitHub profile analyzer. " \
"Your job is to take the provided GitHub profile or repository URL and generate a clear, structured summary covering these points: " \
"1️⃣ **Profile Summary** " \
"- Username " \
"- Bio (if available) " \
"- Total public repositories " \
"- Total followers " \
"- Total stars received (sum across repos) " \
"- Top programming languages (by repo count) " \
"2️⃣ **Repository Highlights** (top 3 by stars or activity) " \
"For each: " \
"- Repository name " \
"- Description " \
"- Primary language " \
"- Star count " \
"- Last updated date " \
"- Notable technologies or frameworks used " \
"3️⃣ **Overall Assessment** " \
"- What does this user specialize in? " \
"- Are they more focused on personal projects or collaborations? " \
"- Any standout strengths or skills you notice? " \
"4️⃣ **Recommendations** " \
"- Suggest one area or technology they could explore next to grow. " \
"- Suggest one improvement to make their GitHub profile more appealing. " \
"Be concise, insightful, and encourage the user’s growth. " \
"If some data is missing, state it clearly instead of guessing."


In [15]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a github named {website.title}"
    user_prompt += "\nThe contents of this github is as follows; \
please provide a summary of this website in markdown."
    user_prompt += website.text
    return user_prompt

In [16]:
system_prompt

'You are a skilled GitHub profile analyzer. Your job is to take the provided GitHub profile or repository URL and generate a clear, structured summary covering these points: 1️⃣ **Profile Summary** - Username - Bio (if available) - Total public repositories - Total followers - Total stars received (sum across repos) - Top programming languages (by repo count) 2️⃣ **Repository Highlights** (top 3 by stars or activity) For each: - Repository name - Description - Primary language - Star count - Last updated date - Notable technologies or frameworks used 3️⃣ **Overall Assessment** - What does this user specialize in? - Are they more focused on personal projects or collaborations? - Any standout strengths or skills you notice? 4️⃣ **Recommendations** - Suggest one area or technology they could explore next to grow. - Suggest one improvement to make their GitHub profile more appealing. Be concise, insightful, and encourage the user’s growth. If some data is missing, state it clearly instead 

In [17]:
print(user_prompt_for(github))

You are looking at a github named Fikriraihan · GitHub
The contents of this github is as follows; please provide a summary of this website in markdown.Skip to content
Navigation Menu
Toggle navigation
Sign in
Appearance settings
Product
GitHub Copilot
Write better code with AI
GitHub Models
New
Manage and compare prompts
GitHub Advanced Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
Why GitHub
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries
View all solutions
Resources
Topics
AI
DevOps
Security
Software Development
View all
Explore
Learning Pathways
Events & Webinars
Ebooks 

In [18]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "tell me a fruit that has red color"}
]

In [19]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, I don’t know, how about the classic red apple? Or maybe you were hoping for something more exotic, like a blood orange? There’s also the ever-popular strawberry. The options are endless! What’s next, a fruit quiz?


In [22]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
messages_for(github)

[{'role': 'system',
  'content': 'You are a skilled GitHub profile analyzer. Your job is to take the provided GitHub profile or repository URL and generate a clear, structured summary covering these points: 1️⃣ **Profile Summary** - Username - Bio (if available) - Total public repositories - Total followers - Total stars received (sum across repos) - Top programming languages (by repo count) 2️⃣ **Repository Highlights** (top 3 by stars or activity) For each: - Repository name - Description - Primary language - Star count - Last updated date - Notable technologies or frameworks used 3️⃣ **Overall Assessment** - What does this user specialize in? - Are they more focused on personal projects or collaborations? - Any standout strengths or skills you notice? 4️⃣ **Recommendations** - Suggest one area or technology they could explore next to grow. - Suggest one improvement to make their GitHub profile more appealing. Be concise, insightful, and encourage the user’s growth. If some data is m

In [24]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [25]:
summarize("https://github.com/Fikriraihan")

'# GitHub Profile Summary for Fikriraihan\n\n### 1️⃣ Profile Summary\n- **Username:** Fikriraihan\n- **Bio:** (No bio available)\n- **Total public repositories:** 34\n- **Total followers:** 0 (not indicated)\n- **Total stars received:** 0\n- **Top programming languages (by repo count):**\n  - JavaScript\n  - TypeScript\n\n### 2️⃣ Repository Highlights\n**Top 3 repositories by activity:**\n\n1. **Repository Name:** 2024-coding-challenge\n   - **Description:** Repository for Coding Challenge 2024\n   - **Primary Language:** JavaScript\n   - **Star Count:** 0\n   - **Last Updated Date:** (Not available)\n   - **Notable Technologies or Frameworks Used:** JavaScript\n\n2. **Repository Name:** ChatGPT\n   - **Description:** (No description provided)\n   - **Primary Language:** TypeScript\n   - **Star Count:** 0\n   - **Last Updated Date:** (Not available)\n   - **Notable Technologies or Frameworks Used:** TypeScript\n\n3. **Repository Name:** fikri-3d-portofolio\n   - **Description:** (No de

In [26]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
display_summary("https://github.com/Fikriraihan")

# GitHub Profile Summary for Fikriraihan

### 1️⃣ Profile Summary
- **Username:** Fikriraihan
- **Bio:** Not available
- **Total public repositories:** 34
- **Total followers:** Not available
- **Total stars received:** 0
- **Top programming languages (by repo count):**
  - JavaScript
  - TypeScript

### 2️⃣ Repository Highlights
Here are the top repositories based on their details:

1. **Repository Name:** 2024-coding-challenge
   - **Description:** Repository for Coding Challenge 2024
   - **Primary Language:** JavaScript
   - **Star Count:** 0
   - **Last Updated Date:** Not available
   - **Notable Technologies/Frameworks Used:** None specified

2. **Repository Name:** ChatGPT
   - **Description:** Not available
   - **Primary Language:** TypeScript
   - **Star Count:** 0
   - **Last Updated Date:** Not available
   - **Notable Technologies/Frameworks Used:** None specified

3. **Repository Name:** fikri-3d-portofolio
   - **Description:** Not available
   - **Primary Language:** JavaScript
   - **Star Count:** 0
   - **Last Updated Date:** Not available
   - **Notable Technologies/Frameworks Used:** None specified

### 3️⃣ Overall Assessment
- **What does this user specialize in?** Fikriraihan specializes in JavaScript and TypeScript, indicating a focus on web development or applications that utilize these languages.
- **Are they more focused on personal projects or collaborations?** The presence of multiple repositories suggests a mix of personal projects. There is no indication of collaboration, as there are no mentions of contributions to external repositories.
- **Any standout strengths or skills you notice?** The variety of repositories shows an interest in different coding challenges and portfolio projects. However, the lack of stars suggests that the projects may not yet attract a significant audience.

### 4️⃣ Recommendations
- **Suggest one area or technology they could explore next to grow:** Given the user’s focus on JavaScript and TypeScript, exploring frameworks like React, Vue.js, or even server-side technologies such as Node.js could be beneficial.
- **Suggest one improvement to make their GitHub profile more appealing:** Adding a bio with a brief introduction and interests, along with project descriptions, would provide more context about the user and enhance engagement with their repositories. Additionally, increasing the visibility of the repositories through more optimization and possibly sharing or collaborating on projects could attract more stars and followers.